In [140]:
import pandas as pd
import numpy as np


In [4]:
##TRAIN UR DATASET
data = pd.read_csv("data.csv")
theta0 = 0
theta1 = 0

In [5]:
data

,km,price
0,240000,3650
1,139800,3800
2,150500,4400
3,185530,4450
4,176000,5250
5,114800,5350
6,166800,5800
7,89000,5990
8,144500,5999
9,84000,6200


In [137]:
kmmean = np.mean(data['km'])/1000
pricemean = np.mean(data['price'])/1000
kmmean,pricemean

(101.06625, 6.331833333333333)

In [111]:
data['kilo/price_cov'] = round((data['km']/1000 - kmmean) * (data['price']/1000-pricemean))
data['kilo_disp'] = round((data['km']/1000 - kmmean) ** 2)

In [134]:
theta0 = data['kilo/price_cov'].sum()/data['kilo_disp'].sum()
theta1 = pricemean - (theta0 * kmmean)
theta0,theta1


(-0.02145297270191494, 8.4688955401636)

In [138]:
#pred = (theta0+theta1*data['km'])/1000

In [157]:
#PREDICTOR
x = int(input("Type kilo here "))
pred = (theta0+theta1*x)/1000
print("prediction: ",pred)

Type kilo here 250000
prediction:  2117.2238635879276
